# QualiCharge maps

## Introduction

L'utilisation de cartes choroplèthe pour la restitution d'indicateurs QualiCharge nécessite le chargement préalable des contours des zones affichées regroupées par cartes.

Pour des raisons de facilité de consultation, les cartes utilisées sont séparées par zone géographique:

- Territoire européen (régions européennes : 11, 24, 27, 28, 32, 44, 52, 53, 75, 76, 84, 93, 94)
- Territoire américain (régions Guadeloupe, Martinique, Guyane : 1, 2, 3)
- Territoire africain (régions La Réunion, Mayotte : 4, 6)

Quatre niveaux de découpage sont proposés:

- régions
- départements
- communes
- agglomérations

Le quatrième niveau est dérivé du découpage communal en intégrant les regroupements de communes définis dans les établissement public de coopération intercommunale (EPCI) à ficalité propre (métropoles, communautés urbaines, communautés d'agglomération, communautés de communes, Aix-Marseille et Grand Paris).

Les régions sont découpées en départements. Les régions ultramarines (hors territoire européen) ne comportent qu'un seul département (code '97x' avec 'x' code région).
Les départements sont découpés en communes.
Les agglomérations sont des regroupements de communes


## Map utilisées

Pour des raisons de lisibilité, les cartes associées à l'ensemble des territoires, au territoire américain et au territoire africain ne sont pas retenues.

Les cartes utilisées pour restituer les indicateurs sont alors les suivantes (la métropole correspond au territoire européen):

- Territoire:
    - métropole par régions
    - métropole par départements
- 18 régions (pour chaque région):
    - région par départements
    - région par ECPI
    - région par commune


## Génération des fichiers

Les fichiers GeoJson pour chaque map sont extraits des [contours administratifs Etalab](https://etalab-datasets.geo.data.gouv.fr/contours-administratifs/2024/geojson/). 
Les régions sont identifiées par leur code [ISO 3166-2:FR](https://fr.wikipedia.org/wiki/ISO_3166-2:FR#R%C3%A9gions_m%C3%A9tropolitaines) :

  - metropole_reg
  - metropole_dep
  - region_xxx_dep
  - region_xxx_ecpi
  - region_xxx_com

  avec xxx : trois caractères région du code ISO 3166-2:FR (ex. ARA pour Auvergne Rhone Alpes)

La précision retenue est de 100 m.

In [3]:
import os
from sqlalchemy import create_engine

engine = create_engine(os.getenv("DATABASE_URL"))

In [32]:
import pandas as pd
query_dep = """SELECT department.code, department.name, ST_AsGeoJSON(department.geometry) :: json -> 'coordinates'  AS polygon, region.code AS reg_code
           FROM department INNER JOIN region ON department.region_id = region.id 
           WHERE region.name = 'Auvergne-Rhône-Alpes'"""
with engine.connect() as conn:
    df = pd.read_sql_query(query_dep, conn)

maps_dir = './maps/'
reg = '84'
file_dep = 'region_' + reg + '_dep.json'
contour = {'type': 'FeatureCollection', 
           'features': [{'type': 'Feature', 
                         'properties': {'code': row[1], 'nom': row[2]}, 
                         'geometry': {'type': 'MultiPolygon', 'coordinates': row[3]}} 
                        for row in list(df.itertuples()) if row[4] == reg]}
with open(maps_dir + file_dep, 'w', encoding ='utf8') as map_file:
    json.dump(contour, map_file)
# print(contour)
# {'properties': {'code': '01', 'nom': 'Ain'}}

In [ ]:
data = requests.get(source_dir + 'epci-100m.geojson')
jsn = json.loads(data.content)
print([feature['properties'] for feature in jsn['features']])

In [22]:
import json 
import requests

reg_metro = ['11', '24', '27', '28', '32', '44', '52', '53', '75', '76', '84', '93', '94']
dpt_metro = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24', '25', '26', '27', '28', '29',
             '2A', '2B', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
             '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', 
             '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95']
# ara_dep = 
source_dir = 'https://etalab-datasets.geo.data.gouv.fr/contours-administratifs/2024/geojson/'
maps_dir = './maps/'

# une seule map pour l'instant -> à compléter
maps = [ ['regions-100m.geojson', reg_metro, 'metropole_reg.json'],
         ['departements-100m.geojson', dpt_metro, 'metropole_dep.json']
       ]
for map in maps:
    data = requests.get(source_dir + map[0])
    jsn = json.loads(data.content)
    # print([feature['properties']['code'] for feature in jsn['features']])
    print(jsn['features'][0])
    contour = {'type': 'FeatureCollection', 'features': [feature for feature in jsn['features'] if feature['properties']['code'] in map[1]]}
    #print([feature['properties']['code'] for feature in contour['features']])
    with open(maps_dir + map[2], 'w', encoding ='utf8') as map_file:
        json.dump(contour, map_file)


{'type': 'Feature', 'properties': {'code': '01', 'nom': 'Guadeloupe'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-61.549, 16.2531], [-61.5466, 16.2545], [-61.5466, 16.2586], [-61.5473, 16.2604], [-61.5476, 16.2648], [-61.5466, 16.2671], [-61.5483, 16.2681], [-61.5475, 16.2702], [-61.5488, 16.2721], [-61.5479, 16.2757], [-61.5487, 16.2773], [-61.5479, 16.2799], [-61.5506, 16.2849], [-61.55, 16.2892], [-61.5462, 16.2929], [-61.5441, 16.292], [-61.542, 16.2879], [-61.5412, 16.2896], [-61.5415, 16.2936], [-61.5425, 16.2961], [-61.544, 16.2953], [-61.5471, 16.2989], [-61.547, 16.3012], [-61.5437, 16.3009], [-61.5412, 16.3025], [-61.5376, 16.3078], [-61.5369, 16.311], [-61.5346, 16.3126], [-61.5351, 16.3144], [-61.5336, 16.3162], [-61.5359, 16.3185], [-61.5342, 16.3226], [-61.5344, 16.3247], [-61.5335, 16.3266], [-61.535, 16.3287], [-61.5356, 16.3314], [-61.5344, 16.3337], [-61.5279, 16.3383], [-61.5297, 16.3393], [-61.5247, 16.3421], [-61.5227, 16.3454], [-61.5199, 16.3432], [

## Utilisation des fichiers

Dans le menu `Admin settings / Maps / add a map` de Metabase, les informations suivantes sont à documenter:

- name (nom de la map),
- URL du fichier (geojson),
- id (lien avec les données à représenter - property geojson),
- name (nom affiché - property geojson)

exemple : https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/regions-version-simplifiee.geojson (property_id = code_region)